In [163]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV 
from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
import datetime
%matplotlib inline


In [5]:
s_2015 = pd.read_csv("data/input_2015.csv")
s_2016 = pd.read_csv("data/input_2016.csv")

In [113]:
def process(data):  
    data = data.drop("Unnamed: 0", axis = 1)
    data = data.reset_index(drop = 1)
    data["date"] = pd.to_datetime(data["date"])
    data["mon"] = data["date"].apply(lambda x: x.month)
    data["day"] = data["date"].apply(lambda x: x.day)
    data["time"] = data["date"].apply(lambda x: x.hour)
    data["dow"] = data["date"].dt.dayofweek
    data["night"] = data["time"].apply(lambda x: 1 if x >= 19 else 0)
    data["weekday"] = data["dow"].apply(lambda x: 1 if (x>=0) & (x<=4) else 0)
    data["April"] = data["date"].apply(lambda x: 1 if x.month == 4 else 0)
    data["May"] = data["date"].apply(lambda x: 1 if x.month == 5 else 0)
    data["June"] = data["date"].apply(lambda x: 1 if x.month == 6 else 0)
    data["July"] = data["date"].apply(lambda x: 1 if x.month == 7 else 0)
    data["August"] = data["date"].apply(lambda x: 1 if x.month == 8 else 0)
    data["September"] = data["date"].apply(lambda x: 1 if x.month == 9 else 0)
    #data["visit_code"] = data["guest"].apply(lambda x: refer[x])
    #data["home_code"] = data["home"].apply(lambda x: refer[x])
    
    data["attendace_ratio"] = data.attendance / data.capacity
    return data

In [141]:
train = process(s_2015)
test = process(s_2016)

In [152]:
train = train[train["date"] >= datetime.date(2015, 10, 27)]
test = test[test["date"] >= datetime.date(2016, 10, 25)]

In [155]:
refer = dict(zip(pd.concat([train["home"], train["guest"],
                           test["home"], test["home"]]), 
               pd.Categorical(pd.concat([train["home"], train["guest"],
                           test["home"], test["home"]])).codes)) ## team code
pos = dict(zip(pd.concat([s_2015["game_center"], s_2016["game_center"]]), 
               pd.Categorical(pd.concat([s_2015["game_center"], s_2016["game_center"]])).codes))

In [156]:
def trans_label(data):
    data["pos"] = data["game_center"].apply(lambda x: pos[x])
    data["visit_code"] = data["guest"].apply(lambda x: refer[x])
    data["home_code"] = data["home"].apply(lambda x: refer[x])
    return data

In [167]:
train = trans_label(train)
test = trans_label(test)

In [38]:
Xnames = ['capacity', 'streak_home', 'streak_guest', 'mon',
       'day', 'time', 'dow', 'night', 'weekday', 'April', 'May', 'June',
       'July', 'August', 'September', 'pos', 'home_code', 'visit_code']

In [79]:
subX = ['capacity', 'streak_home', 'streak_guest',
       'mon', 'day', 'time', 'dow', 'pos', 'home_code', 'visit_code']

In [168]:
Xtrain = train[Xnames]
Xtest = test[Xnames]
y_train = train["attendace_ratio"]
y_test = test["attendace_ratio"]
log_y_train = np.log(y_train+1)
log_y_test = np.log(y_test+1)

In [ ]:
rfr = RandomForestRegressor()
params = {
    "max_depth":range(10, 200, 10),
    "max_features":["auto", "sqrt", "log2"],
    "n_estimators":range(100, 1000, 50),
}
cv_rf = GridSearchCV(estimator=rfr, param_grid=params,
                     scoring="r2", n_jobs = -1, cv=5, verbose=5)
cv_rf.fit(Xtrain, y_train)

Fitting 5 folds for each of 1026 candidates, totalling 5130 fits
[CV] max_depth=10, max_features=auto, n_estimators=100 ...............
[CV] max_depth=10, max_features=auto, n_estimators=100 ...............
[CV] max_depth=10, max_features=auto, n_estimators=100 ...............
[CV] max_depth=10, max_features=auto, n_estimators=100 ...............
[CV] max_depth=10, max_features=auto, n_estimators=100 ...............
[CV] max_depth=10, max_features=auto, n_estimators=150 ...............
[CV] max_depth=10, max_features=auto, n_estimators=150 ...............
[CV] max_depth=10, max_features=auto, n_estimators=150 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=100, score=0.678086, total=   0.5s
[CV]  max_depth=10, max_features=auto, n_estimators=100, score=0.574785, total=   0.5s
[CV] max_depth=10, max_features=auto, n_estimators=150 ...............
[CV] max_depth=10, max_features=auto, n_estimators=150 ...............
[CV]  max_depth=10, max_features=auto, n_estimators

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s


[CV]  max_depth=10, max_features=auto, n_estimators=150, score=0.567782, total=   0.8s
[CV] max_depth=10, max_features=auto, n_estimators=200 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=150, score=0.683024, total=   0.8s
[CV] max_depth=10, max_features=auto, n_estimators=200 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=150, score=0.598450, total=   0.8s
[CV] max_depth=10, max_features=auto, n_estimators=250 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=150, score=0.650366, total=   0.8s
[CV] max_depth=10, max_features=auto, n_estimators=250 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=150, score=0.645953, total=   0.8s
[CV] max_depth=10, max_features=auto, n_estimators=250 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=200, score=0.592771, total=   1.0s
[CV] max_depth=10, max_features=auto, n_estimators=250 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=

[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   13.9s


[CV]  max_depth=10, max_features=auto, n_estimators=650, score=0.689136, total=   3.0s
[CV] max_depth=10, max_features=auto, n_estimators=700 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=650, score=0.595212, total=   3.0s
[CV] max_depth=10, max_features=auto, n_estimators=750 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=650, score=0.636532, total=   2.9s
[CV] max_depth=10, max_features=auto, n_estimators=750 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=650, score=0.652366, total=   2.9s
[CV] max_depth=10, max_features=auto, n_estimators=750 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=700, score=0.569323, total=   3.2s
[CV] max_depth=10, max_features=auto, n_estimators=750 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=700, score=0.690133, total=   3.2s
[CV] max_depth=10, max_features=auto, n_estimators=750 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=

...

[CV]  max_depth=20, max_features=auto, n_estimators=150, score=0.682828, total=   0.9s
[CV] max_depth=20, max_features=auto, n_estimators=200 ...............
[CV]  max_depth=20, max_features=auto, n_estimators=150, score=0.597288, total=   0.9s
[CV] max_depth=20, max_features=auto, n_estimators=200 ...............
[CV]  max_depth=20, max_features=auto, n_estimators=150, score=0.569138, total=   1.1s
[CV] max_depth=20, max_features=auto, n_estimators=200 ...............
[CV]  max_depth=20, max_features=auto, n_estimators=150, score=0.639766, total=   0.9s
[CV] max_depth=20, max_features=auto, n_estimators=200 ...............
[CV]  max_depth=20, max_features=auto, n_estimators=150, score=0.648642, total=   0.9s
[CV] max_depth=20, max_features=auto, n_estimators=250 ...............
[CV]  max_depth=10, max_features=log2, n_estimators=950, score=0.636667, total=   2.9s
[CV] max_depth=20, max_features=auto, n_estimators=250 ...............
[CV]  max_depth=20, max_features=auto, n_estimators=

[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  2.1min


[CV]  max_depth=20, max_features=sqrt, n_estimators=800, score=0.638910, total=   2.9s
[CV] max_depth=20, max_features=sqrt, n_estimators=900 ...............
[CV]  max_depth=20, max_features=sqrt, n_estimators=850, score=0.569801, total=   3.0s
[CV] max_depth=20, max_features=sqrt, n_estimators=900 ...............
[CV]  max_depth=20, max_features=sqrt, n_estimators=850, score=0.676539, total=   3.0s
[CV] max_depth=20, max_features=sqrt, n_estimators=900 ...............
[CV]  max_depth=20, max_features=sqrt, n_estimators=850, score=0.641320, total=   3.0s
[CV] max_depth=20, max_features=sqrt, n_estimators=950 ...............
[CV]  max_depth=20, max_features=sqrt, n_estimators=850, score=0.648511, total=   2.9s
[CV] max_depth=20, max_features=sqrt, n_estimators=950 ...............
[CV]  max_depth=20, max_features=sqrt, n_estimators=850, score=0.641713, total=   2.9s
[CV] max_depth=20, max_features=sqrt, n_estimators=950 ...............
[CV]  max_depth=20, max_features=sqrt, n_estimators=

[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.2min


[CV]  max_depth=30, max_features=auto, n_estimators=950, score=0.563781, total=   5.8s
[CV] max_depth=30, max_features=sqrt, n_estimators=200 ...............
[CV]  max_depth=30, max_features=sqrt, n_estimators=150, score=0.649282, total=   0.6s
[CV] max_depth=30, max_features=sqrt, n_estimators=200 ...............
[CV]  max_depth=30, max_features=sqrt, n_estimators=150, score=0.634417, total=   0.6s
[CV] max_depth=30, max_features=sqrt, n_estimators=200 ...............
[CV]  max_depth=30, max_features=sqrt, n_estimators=200, score=0.571264, total=   0.8s
[CV] max_depth=30, max_features=sqrt, n_estimators=200 ...............
[CV]  max_depth=30, max_features=auto, n_estimators=950, score=0.685509, total=   5.8s
[CV] max_depth=30, max_features=sqrt, n_estimators=250 ...............
[CV]  max_depth=30, max_features=sqrt, n_estimators=200, score=0.682330, total=   0.8s
[CV] max_depth=30, max_features=sqrt, n_estimators=250 ...............
[CV]  max_depth=30, max_features=sqrt, n_estimators=

In [160]:
cv_rf.best_score_

0.6451543085384811

In [75]:
rf_best = RandomForestRegressor(max_depth=190, max_features="auto", n_estimators=350).fit(Xtrain, y_train)

In [161]:
cv_rf.best_estimator_.score(Xtrain, y_train)

0.9529943721606412

In [162]:
cv_rf.best_estimator_.score(Xtest, y_test)

0.61108897187795674

In [81]:
Xsub_train = train[subX]
Xsub_test = test[subX]

In [82]:
rfr = RandomForestRegressor()
params = {
    "max_depth":range(10, 200, 10),
    "max_features":["auto", "sqrt", "log2"],
    "n_estimators":range(100, 1000, 50),
}
sub_model = GridSearchCV(estimator=rfr, param_grid=params,
                     scoring="r2", n_jobs = -1, cv=5, verbose=5)
sub_model.fit(Xsub_train, y_train)

Fitting 5 folds for each of 1026 candidates, totalling 5130 fits
[CV] max_depth=10, max_features=auto, n_estimators=100 ...............
[CV] max_depth=10, max_features=auto, n_estimators=100 ...............
[CV] max_depth=10, max_features=auto, n_estimators=100 ...............
[CV] max_depth=10, max_features=auto, n_estimators=100 ...............
[CV] max_depth=10, max_features=auto, n_estimators=100 ...............
[CV] max_depth=10, max_features=auto, n_estimators=150 ...............
[CV] max_depth=10, max_features=auto, n_estimators=150 ...............
[CV] max_depth=10, max_features=auto, n_estimators=150 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=100, score=-1.099344, total=   0.6s
[CV] max_depth=10, max_features=auto, n_estimators=150 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=100, score=0.630504, total=   0.6s
[CV]  max_depth=10, max_features=auto, n_estimators=100, score=0.555701, total=   0.6s
[CV] max_depth=10, max_features=au

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.7s


[CV]  max_depth=10, max_features=auto, n_estimators=150, score=-0.934076, total=   1.0s
[CV] max_depth=10, max_features=auto, n_estimators=200 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=150, score=0.552725, total=   1.0s
[CV]  max_depth=10, max_features=auto, n_estimators=150, score=0.628164, total=   1.0s
[CV] max_depth=10, max_features=auto, n_estimators=200 ...............
[CV] max_depth=10, max_features=auto, n_estimators=250 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=150, score=0.688313, total=   1.0s
[CV] max_depth=10, max_features=auto, n_estimators=250 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=150, score=0.327484, total=   1.0s
[CV] max_depth=10, max_features=auto, n_estimators=250 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=200, score=-0.852394, total=   1.3s
[CV]  max_depth=10, max_features=auto, n_estimators=200, score=0.628525, total=   1.2s
[CV] max_depth=10, max_features=au

[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   17.5s


[CV]  max_depth=10, max_features=auto, n_estimators=650, score=0.631345, total=   3.7s
[CV] max_depth=10, max_features=auto, n_estimators=700 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=650, score=0.563398, total=   3.6s
[CV] max_depth=10, max_features=auto, n_estimators=750 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=650, score=0.681245, total=   3.7s
[CV] max_depth=10, max_features=auto, n_estimators=750 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=650, score=0.325123, total=   3.7s
[CV] max_depth=10, max_features=auto, n_estimators=750 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=700, score=-0.972321, total=   4.0s
[CV] max_depth=10, max_features=auto, n_estimators=750 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=700, score=0.633085, total=   3.9s
[CV] max_depth=10, max_features=auto, n_estimators=750 ...............
[CV]  max_depth=10, max_features=auto, n_estimators

...

[CV]  max_depth=150, max_features=log2, n_estimators=800, score=0.318515, total=   3.9s
[CV] max_depth=150, max_features=log2, n_estimators=900 ..............
[CV]  max_depth=150, max_features=log2, n_estimators=850, score=0.438202, total=   4.2s
[CV] max_depth=150, max_features=log2, n_estimators=900 ..............
[CV]  max_depth=150, max_features=log2, n_estimators=850, score=0.675328, total=   4.2s
[CV] max_depth=150, max_features=log2, n_estimators=900 ..............
[CV]  max_depth=150, max_features=log2, n_estimators=850, score=0.617028, total=   4.4s
[CV] max_depth=150, max_features=log2, n_estimators=950 ..............
[CV]  max_depth=150, max_features=log2, n_estimators=850, score=0.668612, total=   4.2s
[CV] max_depth=150, max_features=log2, n_estimators=950 ..............
[CV]  max_depth=150, max_features=log2, n_estimators=850, score=0.316690, total=   4.1s
[CV] max_depth=150, max_features=log2, n_estimators=950 ..............
[CV]  max_depth=150, max_features=log2, n_esti

[Parallel(n_jobs=-1)]: Done 4592 tasks      | elapsed: 31.0min


[CV]  max_depth=170, max_features=log2, n_estimators=950, score=0.614020, total=   4.9s
[CV] max_depth=180, max_features=auto, n_estimators=200 ..............
[CV]  max_depth=180, max_features=auto, n_estimators=150, score=0.603337, total=   1.2s
[CV] max_depth=180, max_features=auto, n_estimators=200 ..............
[CV]  max_depth=180, max_features=auto, n_estimators=150, score=0.566246, total=   1.2s
[CV] max_depth=180, max_features=auto, n_estimators=200 ..............
[CV]  max_depth=180, max_features=auto, n_estimators=150, score=0.676105, total=   1.2s
[CV] max_depth=180, max_features=auto, n_estimators=200 ..............
[CV]  max_depth=170, max_features=log2, n_estimators=950, score=0.673513, total=   4.8s
[CV] max_depth=180, max_features=auto, n_estimators=250 ..............
[CV]  max_depth=180, max_features=auto, n_estimators=150, score=0.334832, total=   1.2s
[CV] max_depth=180, max_features=auto, n_estimators=250 ..............
[CV]  max_depth=180, max_features=auto, n_esti

[Parallel(n_jobs=-1)]: Done 5130 out of 5130 | elapsed: 34.8min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': range(10, 200, 10), 'max_features': ['auto', 'sqrt', 'log2'], 'n_estimators': range(100, 1000, 50)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=5)

In [70]:
rfr = RandomForestRegressor()
params = {
    "max_depth":range(10, 200, 10),
    "max_features":["auto", "sqrt", "log2"],
    "n_estimators":range(100, 1000, 50),
}
cv_rf = GridSearchCV(estimator=rfr, param_grid=params,
                     scoring="r2", n_jobs = -1, cv=5, verbose=5)
cv_rf.fit(Xtrain, log_y_train)

Fitting 5 folds for each of 1026 candidates, totalling 5130 fits
[CV] max_depth=10, max_features=auto, n_estimators=100 ...............
[CV] max_depth=10, max_features=auto, n_estimators=100 ...............
[CV] max_depth=10, max_features=auto, n_estimators=100 ...............
[CV] max_depth=10, max_features=auto, n_estimators=100 ...............
[CV] max_depth=10, max_features=auto, n_estimators=100 ...............
[CV] max_depth=10, max_features=auto, n_estimators=150 ...............
[CV] max_depth=10, max_features=auto, n_estimators=150 ...............
[CV] max_depth=10, max_features=auto, n_estimators=150 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=100, score=-1.786987, total=   0.5s
[CV] max_depth=10, max_features=auto, n_estimators=150 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=100, score=0.590784, total=   0.5s
[CV]  max_depth=10, max_features=auto, n_estimators=100, score=0.490440, total=   0.5s
[CV] max_depth=10, max_features=au

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s


[CV]  max_depth=10, max_features=auto, n_estimators=150, score=-1.607526, total=   0.8s
[CV] max_depth=10, max_features=auto, n_estimators=200 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=150, score=0.489605, total=   0.8s
[CV] max_depth=10, max_features=auto, n_estimators=200 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=150, score=0.592231, total=   0.9s
[CV] max_depth=10, max_features=auto, n_estimators=250 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=150, score=0.285088, total=   0.8s
[CV] max_depth=10, max_features=auto, n_estimators=250 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=150, score=0.686324, total=   0.9s
[CV] max_depth=10, max_features=auto, n_estimators=250 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=200, score=0.594322, total=   1.1s
[CV]  max_depth=10, max_features=auto, n_estimators=200, score=-1.132707, total=   1.1s
[CV] max_depth=10, max_features=au

[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   14.4s


[CV]  max_depth=10, max_features=auto, n_estimators=650, score=0.612390, total=   3.1s
[CV] max_depth=10, max_features=auto, n_estimators=700 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=650, score=0.522892, total=   3.1s
[CV] max_depth=10, max_features=auto, n_estimators=750 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=650, score=0.675033, total=   3.0s
[CV] max_depth=10, max_features=auto, n_estimators=750 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=650, score=0.290478, total=   3.0s
[CV] max_depth=10, max_features=auto, n_estimators=750 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=700, score=-1.349591, total=   3.3s
[CV] max_depth=10, max_features=auto, n_estimators=750 ...............
[CV]  max_depth=10, max_features=auto, n_estimators=700, score=0.600278, total=   3.3s
[CV] max_depth=10, max_features=auto, n_estimators=750 ...............
[CV]  max_depth=10, max_features=auto, n_estimators

...

[CV]  max_depth=150, max_features=log2, n_estimators=800, score=0.263894, total=   2.9s
[CV] max_depth=150, max_features=log2, n_estimators=900 ..............
[CV]  max_depth=150, max_features=log2, n_estimators=850, score=0.452910, total=   3.2s
[CV] max_depth=150, max_features=log2, n_estimators=900 ..............
[CV]  max_depth=150, max_features=log2, n_estimators=850, score=0.636299, total=   3.2s
[CV] max_depth=150, max_features=log2, n_estimators=900 ..............
[CV]  max_depth=150, max_features=log2, n_estimators=850, score=0.608343, total=   3.2s
[CV] max_depth=150, max_features=log2, n_estimators=950 ..............
[CV]  max_depth=150, max_features=log2, n_estimators=850, score=0.655324, total=   3.2s
[CV] max_depth=150, max_features=log2, n_estimators=950 ..............
[CV]  max_depth=150, max_features=log2, n_estimators=850, score=0.267476, total=   3.1s
[CV] max_depth=150, max_features=log2, n_estimators=950 ..............
[CV]  max_depth=150, max_features=log2, n_esti

[Parallel(n_jobs=-1)]: Done 4592 tasks      | elapsed: 23.9min


[CV] max_depth=180, max_features=auto, n_estimators=200 ..............
[CV]  max_depth=180, max_features=auto, n_estimators=150, score=0.573933, total=   1.0s
[CV] max_depth=180, max_features=auto, n_estimators=200 ..............
[CV]  max_depth=180, max_features=auto, n_estimators=150, score=0.512557, total=   1.0s
[CV] max_depth=180, max_features=auto, n_estimators=200 ..............
[CV]  max_depth=180, max_features=auto, n_estimators=150, score=0.670188, total=   1.0s
[CV] max_depth=180, max_features=auto, n_estimators=200 ..............
[CV]  max_depth=170, max_features=log2, n_estimators=950, score=0.658544, total=   3.6s
[CV] max_depth=180, max_features=auto, n_estimators=250 ..............
[CV]  max_depth=180, max_features=auto, n_estimators=150, score=0.293463, total=   1.0s
[CV] max_depth=180, max_features=auto, n_estimators=250 ..............
[CV]  max_depth=170, max_features=log2, n_estimators=950, score=0.265758, total=   3.5s
[CV] max_depth=180, max_features=auto, n_estim

[Parallel(n_jobs=-1)]: Done 5130 out of 5130 | elapsed: 26.8min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': range(10, 200, 10), 'max_features': ['auto', 'sqrt', 'log2'], 'n_estimators': range(100, 1000, 50)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=5)

In [72]:
cv_rf.best_estimator_.score(Xtrain, log_y_train)

0.94469832579269475

In [87]:
y_mean = np.mean(log_y_train)
y_gr_train = log_y_train - y_mean
y_gr_test = log_y_test - y_mean

In [88]:
kernel = ConstantKernel() + Matern(length_scale=2, nu=3/2) + WhiteKernel(noise_level=1)

In [165]:
gpr = gaussian_process.GaussianProcessRegressor(kernel=kernel)
gpr.fit(Xtrain, y_gr_train)

ValueError: Found input variables with inconsistent numbers of samples: [1272, 1351]